In [ ]:
# === 셀 1 패키지 설치 ===
# 노트북에서 바로 실행
%pip -q install pandas numpy scikit-learn matplotlib sentence-transformers rapidfuzz orjson

In [ ]:
# === 셀 2 임포트 및 기본 설정 ===
import os
import re
import json
import orjson
import numpy as np
import pandas as pd
from ast import literal_eval
from collections import Counter, defaultdict

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer, util

pd.set_option("display.max_colwidth", 200)
RNG = np.random.default_rng(42)

# 경로
ICON_CSV = "./output_MMT_icon.csv"
WIDGET_CSV = "widget_captions.csv"

# 모델
EMB_MODEL_NAME = "all-MiniLM-L6-v2"
emb_model = SentenceTransformer(EMB_MODEL_NAME)

# 함수 유틸
def norm_text(s: str) -> str:
    if pd.isna(s):
        return ""
    s = str(s)
    s = s.replace("\n", " ").strip().lower()
    s = re.sub(r"\s+", " ", s)
    return s

def split_pipe(s: str):
    s = norm_text(s)
    if not s:
        return []
    return [t.strip() for t in s.split("|") if t.strip()]

def safe_json_loads(s: str):
    if pd.isna(s) or not str(s).strip():
        return {}
    s = str(s)
    # 기본 json 혹은 json과 유사한 문자열 케이스 모두 시도
    try:
        return json.loads(s)
    except Exception:
        try:
            return orjson.loads(s)
        except Exception:
            # 내부 쌍따옴표 이스케이프 이슈 보정
            try:
                s2 = s.replace('""', '"')
                return json.loads(s2)
            except Exception:
                return {}


In [ ]:
# === 셀 3 데이터 로드 및 전처리 ===
# output_MMT_icon.csv 포맷 가정
df_icon = pd.read_csv(ICON_CSV)
# widget_captions.csv 포맷 가정
df_widget = pd.read_csv(WIDGET_CSV)

# 핵심 컬럼 정리
need_cols_icon = ["screenId", "icon_path", "bounds", "reference_captions", "icon_context", "generated_caption"]
for c in need_cols_icon:
    if c not in df_icon.columns:
        raise ValueError(f"output_MMT_icon.csv에 {c} 컬럼이 없음")

need_cols_widget = ["datasetId", "screenId", "nodeId", "captions"]
for c in need_cols_widget:
    if c not in df_widget.columns:
        raise ValueError(f"widget_captions.csv에 {c} 컬럼이 없음")

# 정규화
df_icon["generated_caption"] = df_icon["generated_caption"].map(norm_text)
df_icon["reference_list"] = df_icon["reference_captions"].map(split_pipe)
df_icon["bounds"] = df_icon["bounds"].astype(str)

# icon_context 파싱
df_icon["icon_context_json"] = df_icon["icon_context"].apply(safe_json_loads)
def extract_ctx(d):
    if not isinstance(d, dict):
        return pd.Series({"class_name": None, "resource_id": None, "activity": None})
    ui = d.get("UI element info", {}) or d.get("ui element info", {})
    return pd.Series({
        "class_name": ui.get("class_name"),
        "resource_id": ui.get("resource_id"),
        "activity": d.get("app activity name") or d.get("activity")
    })
ctx = df_icon["icon_context_json"].apply(extract_ctx)
df_icon = pd.concat([df_icon, ctx], axis=1)

# widget 캡션 파싱
df_widget["captions_list"] = df_widget["captions"].map(split_pipe)

print("icon rows", len(df_icon))
print("widget rows", len(df_widget))
df_icon.head(3)


In [ ]:
# === 셀 4 규칙 추출 빈도 분석 ===
# generated_caption 상위 N 집계
cap_counts = df_icon["generated_caption"].value_counts().reset_index()
cap_counts.columns = ["caption", "count"]
top_k = 50
top_captions = cap_counts.head(top_k)
top_captions.head(10)


In [ ]:
# === 셀 5 n-gram 패턴 추출 ===
# 자주 쓰이는 bigram trigram으로 규칙 후보 만들기
texts = df_icon["generated_caption"].fillna("").tolist()
vectorizer = CountVectorizer(ngram_range=(1,3), min_df=5)  # 5회 이상 등장
X = vectorizer.fit_transform(texts)
vocab = np.array(vectorizer.get_feature_names_out())
freq = np.asarray(X.sum(axis=0)).ravel()
ng_df = pd.DataFrame({"ngram": vocab, "freq": freq}).sort_values("freq", ascending=False)
ng_df.head(20)


In [ ]:
# === 셀 6 카테고리 규칙 분류 샘플 ===
# 간단한 정규식 카테고리 분류로 대표 규칙 라벨링
cat_rules = {
    "navigation": re.compile(r"\b(back|go back|previous|home|menu|close|exit)\b"),
    "search": re.compile(r"\b(search|find|look for)\b"),
    "action": re.compile(r"\b(open|toggle|share|edit|delete|save|apply|submit|send)\b"),
    "settings": re.compile(r"\b(setting|preferences|option)\b"),
    "media": re.compile(r"\b(play|pause|stop|record|camera|gallery)\b"),
}

def label_category(caption: str):
    for k, pat in cat_rules.items():
        if pat.search(caption):
            return k
    return "other"

df_icon["rule_category"] = df_icon["generated_caption"].map(label_category)
cat_stats = df_icon["rule_category"].value_counts().reset_index()
cat_stats.columns = ["category", "count"]
cat_stats


In [ ]:
# === 셀 7 BERT 임베딩 기반 유사도 계산 ===
# reference_captions 대비 generated_caption 최대 유사도
def bert_sim_max(gen: str, refs: list[str]) -> float:
    gen = norm_text(gen)
    refs = [norm_text(r) for r in refs if r]
    if not gen or not refs:
        return np.nan
    e_gen = emb_model.encode(gen, normalize_embeddings=True)
    e_ref = emb_model.encode(refs, normalize_embeddings=True)
    sim = util.cos_sim(e_gen, e_ref).cpu().numpy().ravel()
    return float(sim.max()) if len(sim) else np.nan

df_icon["sim_ref_max"] = df_icon.apply(lambda r: bert_sim_max(r["generated_caption"], r["reference_list"]), axis=1)

# 위젯 캡션과 화면 단위로 매칭해 최대 유사도
# 화면별 모든 widget captions를 묶어서 gen과 비교
widget_by_screen = df_widget.groupby("screenId")["captions_list"].apply(list)
# 리스트의 리스트를 펼쳐서 한 화면의 모든 후보 문자열로
widget_by_screen = widget_by_screen.map(lambda lst: [c for sub in lst for c in sub])

def bert_sim_widget(gen: str, screen_id) -> float:
    gen = norm_text(gen)
    if not gen or screen_id not in widget_by_screen:
        return np.nan
    refs = [norm_text(x) for x in widget_by_screen[screen_id] if x]
    if not refs:
        return np.nan
    e_gen = emb_model.encode(gen, normalize_embeddings=True)
    # 메모리 절약을 위해 배치 처리
    sims = []
    batch = 64
    for i in range(0, len(refs), batch):
        e_ref = emb_model.encode(refs[i:i+batch], normalize_embeddings=True)
        s = util.cos_sim(e_gen, e_ref).cpu().numpy().ravel()
        sims.append(s.max())
    return float(np.max(sims)) if sims else np.nan

df_icon["sim_widget_max"] = df_icon.apply(lambda r: bert_sim_widget(r["generated_caption"], r["screenId"]), axis=1)

df_icon[["generated_caption", "reference_list", "sim_ref_max", "sim_widget_max"]].head(10)


In [ ]:
# === 셀 8 규칙 기반 유효성 점수 ===
# 길이 점수 2~7 단어면 최고
def length_score(caption: str):
    w = norm_text(caption).split()
    n = len(w)
    if n == 0:
        return 0.0
    if 2 <= n <= 7:
        return 1.0
    if n == 1:
        return 0.4
    return max(0.4, 1.0 - (n - 7)*0.05)

# 기능성 키워드 점수
FUNC_WORDS = set("""
back previous search find open close menu settings save delete edit share send apply submit next previous play pause camera gallery
""".split())

def functional_score(caption: str):
    w = set(norm_text(caption).split())
    return 1.0 if len(FUNC_WORDS & w) > 0 else 0.0

# 동일 화면 내 중복 페널티 유니크 비율
dup_ratio = df_icon.groupby("screenId")["generated_caption"].apply(
    lambda s: 1.0 - (s.nunique() / max(1, len(s)))
)
# 화면별 동일 캡션 반복이 많으면 점수 내려간다
def uniqueness_score(screen_id):
    r = dup_ratio.get(screen_id, 0.0)
    return float(max(0.0, 1.0 - r))

df_icon["len_score"] = df_icon["generated_caption"].map(length_score)
df_icon["func_score"] = df_icon["generated_caption"].map(functional_score)
df_icon["uniq_score"] = df_icon["screenId"].map(uniqueness_score)

# 최종 유효성 스코어 가중합
# 시맨틱 유사도 0.6 길이 0.15 기능성 0.15 유니크 0.1
sim_ref = df_icon["sim_ref_max"].fillna(0.0)
sim_widget = df_icon["sim_widget_max"].fillna(0.0)
# 참고 기준이 두 가지면 보수적으로 최대값 사용
sim_base = np.maximum(sim_ref, sim_widget)

df_icon["validity_score"] = (
    0.60 * sim_base +
    0.15 * df_icon["len_score"] +
    0.15 * df_icon["func_score"] +
    0.10 * df_icon["uniq_score"]
)

df_icon[["generated_caption", "sim_ref_max", "sim_widget_max", "len_score", "func_score", "uniq_score", "validity_score"]].head(10)


In [ ]:
# === 셀 9 대표 규칙 리포트 ===
# 1 상위 캡션
print("상위 빈도 캡션")
display(top_captions.head(20))

# 2 상위 n-gram
print("상위 n-gram")
display(ng_df.head(20))

# 3 카테고리 분포
print("카테고리 분포")
display(cat_stats)

# 4 클래스 이름별 대표 캡션 및 평균 유효성
by_class = (
    df_icon.groupby(["class_name"])  # None 포함 가능
    .agg(
        n=("generated_caption", "count"),
        uniq_caps=("generated_caption", "nunique"),
        mean_validity=("validity_score", "mean")
    )
    .reset_index()
    .sort_values("n", ascending=False)
)
print("UI class_name 기준 요약")
display(by_class.head(20))

# 5 유효성 분포
bins = [0, 0.4, 0.6, 0.8, 1.0]
labels = ["low", "mid", "good", "excellent"]
df_icon["validity_band"] = pd.cut(df_icon["validity_score"], bins=bins, labels=labels, include_lowest=True)
band_counts = df_icon["validity_band"].value_counts().reindex(labels).fillna(0).astype(int)
print("유효성 밴드 분포")
display(band_counts.to_frame(name="count").T)

# 6 화면 단위 다양성
diversity = df_icon.groupby("screenId")["generated_caption"].agg(lambda s: s.nunique()/max(1,len(s))).rename("diversity_ratio").reset_index()
print("화면 단위 캡션 다양성 상하위 예시")
display(diversity.sort_values("diversity_ratio", ascending=True).head(10))
display(diversity.sort_values("diversity_ratio", ascending=False).head(10))


In [ ]:
# === 셀 10 간단 시각화 ===
# 유효성 분포 히스토그램
plt.figure(figsize=(6,4))
plt.hist(df_icon["validity_score"].dropna(), bins=20)
plt.title("Validity score distribution")
plt.xlabel("score")
plt.ylabel("count")
plt.show()

# 카테고리별 평균 유효성
cat_mean = df_icon.groupby("rule_category")["validity_score"].mean().sort_values(ascending=False)
plt.figure(figsize=(6,4))
cat_mean.plot(kind="bar")
plt.title("Mean validity by rule category")
plt.xlabel("category")
plt.ylabel("mean validity")
plt.show()
